<a href="https://colab.research.google.com/github/Balajiscientist/Balajiscientist-ML-Challenge-2025-Smart-Product-Pricing-Solution/blob/main/Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu128
!pip install unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

Looking in indexes: https://download.pytorch.org/whl/cu128
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 125.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 k

In [4]:
import os
import pandas as pd
import numpy as np

# Additional imports
import re
import multiprocessing
from time import time as timer
from tqdm import tqdm
from pathlib import Path
from functools import partial
import requests
import urllib
import torch
from PIL import Image


train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
sample_test = pd.read_csv('/content/sample_test.csv')
sample_test_out = pd.read_csv('/content/sample_test_out.csv')

In [5]:
import re
import os
import pandas as pd
import multiprocessing
from time import time as timer
from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
import requests
import urllib

def download_image(image_link, save_folder):
    if isinstance(image_link, str):
        filename = Path(image_link).name
        image_save_path = os.path.join(save_folder, filename)
        if not os.path.exists(image_save_path):
            try:
                urllib.request.urlretrieve(image_link, image_save_path)
            except Exception as ex:
                print(f"Warning: Not able to download - {image_link}: {ex}")
        return
    return

def download_images(image_links, download_folder):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)
    results = []
    download_image_partial = partial(download_image, save_folder=download_folder)
    with multiprocessing.Pool(100) as pool:
        for result in tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)):
            results.append(result)
        pool.close()
        pool.join()

In [6]:
# Use first 1000 samples for faster experimentation
train = train.head(1000)
download_images(train["image_link"], "/balaji/train")


100%|██████████| 1000/1000 [00:02<00:00, 337.06it/s]


In [7]:
from PIL import Image

# IMPROVED instruction with better guidance
instruction = f"""
You are a helpful assistant.
The user provides a product catalog and an image. Your task is to predict the price in USD.

Important:
- Do NOT use numbers from weights, volumes, or quantities like "12.7 Ounce", "11.25 oz" as the price.
- Consider the type of product, brand, and image cues to estimate the correct price.
- Look at the product category, brand reputation, packaging quality, and visual appearance.
- Consider typical price ranges for similar products.
- Output ONLY the price as a number in USD.
- Example: 4.89

"""

# NEW: Price preprocessing function
def preprocess_price(price):
    """Normalize price values for better training stability"""
    return min(max(float(price), 0.01), 1000.0)  # Clamp extreme values

def convert_to_conversation(sample):
    image_filename = os.path.basename(sample["image_link"])
    image_path = f"/balaji/train/{image_filename}"
    image = Image.open(image_path).convert("RGB")

    # APPLY price preprocessing - THIS IS NEW
    normalized_price = preprocess_price(sample["price"])

    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": f"{instruction}{sample['catalog_content']}"},
                {"type": "image", "image": image},
            ],
        },
        {
            "role": "assistant",
            "content": [
                {"type": "text", "text": str(normalized_price)},  # Using normalized price
            ],
        }
    ]

    return {"messages": conversation}


In [8]:
converted_dataset = []
for row in train.itertuples(index=False):
    sample = {
        "sample_id": row.sample_id,
        "catalog_content": row.catalog_content,
        "image_link": row.image_link,
        "price": row.price,
    }
    converted_dataset.append(convert_to_conversation(sample))


In [9]:
from PIL import Image

instruction = f"""
You are a helpful assistant.
The user provides a product catalog and an image. Your task is to predict the price in USD.

Important:
- Do NOT use numbers from weights, volumes, or quantities (like '12.7 Ounce', '11.25 oz') as the price.
- Consider the type of product, brand, and image cues to estimate the correct price.
- Output ONLY the price as a number in USD.
- Example: 4.89
"""

def convert_to_conversation(sample):
    image_filename = os.path.basename(sample["image_link"])
    image_path = f"/balaji/train/{image_filename}" # Corrected path
    image = Image.open(image_path).convert("RGB")

    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": f"{instruction}\n\n{sample['catalog_content']}"},
                {"type": "image", "image": image}
            ]
        },
        {
            "role": "assistant",
            "content": [
                {"type": "text", "text": str(sample["price"])}
            ]
        }
    ]

    return {"messages": conversation}

In [10]:
train.loc[0, "catalog_content"]


'Item Name: La Victoria Green Taco Sauce Mild, 12 Ounce (Pack of 6)\nValue: 72.0\nUnit: Fl Oz\n'

In [11]:
converted_dataset = []
for row in train.itertuples(index=False):
    sample = {
        "sample_id": row.sample_id,
        "catalog_content": row.catalog_content,
        "image_link": row.image_link,
        "price": row.price
    }
    converted_dataset.append(convert_to_conversation(sample))

In [12]:
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen2.5-VL-3B-Instruct-bnb-4bit",
    load_in_4bit=True,
    use_gradient_checkpointing="unsloth",
    device_map="auto",  # NEW: Better GPU utilization
    torch_dtype=torch.bfloat16,  # NEW: Explicit dtype
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.1: Fast Qwen2_5_Vl patching. Transformers: 4.55.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


...

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


chat_template.json: 0.00B [00:00, ?B/s]

In [13]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers=True,
    finetune_language_layers=True,
    finetune_attention_modules=True,
    finetune_mlp_modules=True,
    r=64,  # CHANGED: Was 32, now 64 (better adaptation)
    lora_alpha=128,  # CHANGED: Was 32, now 128 (stronger adaptation)
    lora_dropout=0.05,  # CHANGED: Was 0.01, now 0.05 (slightly higher dropout)
    bias="none",
    random_state=3407,
    use_rslora=True,  # CHANGED: Was False, now True (enable RSLoRA)
    loftq_config=None,
)



Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


In [15]:
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=UnslothVisionDataCollator(model, tokenizer),
    train_dataset=converted_dataset,
    args=SFTConfig(
        per_device_train_batch_size=1,  # CHANGED: Was 2, now 1 (better gradients)
        gradient_accumulation_steps=8,  # CHANGED: Was 4, now 8 (maintain effective batch size)
        warmup_steps=50,  # CHANGED: Was 10, now 50 (more warmup)
        max_steps=2000,  # CHANGED: Was 1000, now 2000 (double training steps)
        learning_rate=5e-5,  # CHANGED: Was 1e-4, now 5e-5 (lower for finer adjustments)
        logging_steps=10,  # CHANGED: Was 1, now 10
        optim="adamw_8bit",
        weight_decay=0.02,  # CHANGED: Was 0.01, now 0.02 (higher weight decay)
        lr_scheduler_type="cosine_with_restarts",  # CHANGED: Was "cosine", now with restarts
        # num_cycles=2,  # REMOVED: This argument is not supported by SFTConfig
        seed=3407,
        output_dir="outputs",
        report_to="none",
        remove_unused_columns=False,
        dataset_text_field="",
        dataset_kwargs={"skip_prepare_dataset": True},
        max_length=2048,
        fp16=False,  # NEW: Enable mixed precision
        bf16=True,
        dataloader_num_workers=2,  # NEW: Faster data loading
        save_steps=500,  # NEW: Save checkpoints
        eval_steps=500,  # NEW: Evaluation steps
        save_total_limit=3,  # NEW: Keep only best checkpoints
    ),
)

# Start training
trainer_stats = trainer.train()

Unsloth: Model does not have a default image size - using 512


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 16 | Total steps = 2,000
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 164,339,712 of 3,918,962,688 (4.19% trained)


Step,Training Loss
10,1.157400
20,1.181300
30,1.060300
40,1.152800
50,1.035000
60,1.434900
70,1.238700
80,1.273900
90,1.325700
100,1.688300


Step,Training Loss
10,1.157400
20,1.181300
30,1.060300
40,1.152800
50,1.035000
60,1.434900
70,1.238700
80,1.273900
90,1.325700
100,1.688300


In [16]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
5.225 GB of memory reserved.


In [17]:
model.save_pretrained("/scientistbalaji/working/fine_tuned_model")
tokenizer.save_pretrained("/scientistbalaji/working/fine_tuned_model")
print("Model saved successfully.")

Model saved successfully.


In [18]:
FastVisionModel.for_inference(model)

def predict_price(catalog_content, image_link):
    try:
        image_filename = os.path.basename(image_link)
        image_path = f"/balaji/train/{image_filename}"

        if not os.path.exists(image_path):
            download_image(image_link, "/balaji/train")

        image = Image.open(image_path).convert("RGB")

        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": f"{instruction}{catalog_content}"},
                    {"type": "image", "image": image},
                ],
            }
        ]

        input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

        inputs = tokenizer(
            images=[image],
            text=[input_text],
            add_special_tokens=True,
            return_tensors="pt",
        ).to("cuda")

        # NEW: Multiple inference attempts with different parameters
        attempts = [
            {"temperature": 0.1, "top_p": 0.8, "num_beams": 5},
            {"temperature": 0.3, "top_p": 0.9, "num_beams": 3},
            {"temperature": 0.5, "top_p": 0.95, "num_beams": 1},
        ]

        prices = []
        for attempt in attempts:
            outputs = model.generate(
                **inputs,
                max_new_tokens=15,  # CHANGED: Was 10, now 15
                do_sample=True,
                temperature=attempt["temperature"],
                top_p=attempt["top_p"],
                num_beams=attempt["num_beams"],  # NEW: Beam search
                repetition_penalty=1.1,  # NEW: Prevent repetition
                early_stopping=True,  # NEW: Early stopping
                pad_token_id=tokenizer.eos_token_id,
            )

            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            price_str = generated_text.split("assistant")[-1].strip().replace("<|im_end|>", "").strip()

            try:
                price = abs(float(price_str))
                if 0.01 <= price <= 1000:  # Reasonable price range
                    prices.append(price)
            except:
                continue

        # NEW: Use median of valid predictions
        if prices:
            price = np.median(prices)
        else:
            price = 10.0  # Default fallback

    except Exception as e:
        print(f"Error predicting for {image_link}: {e}")
        price = 10.0

    return price


In [20]:
# Create directory for test images
os.makedirs("/balaji/train", exist_ok=True)

# Download test images
download_images(sample_test["image_link"], "/balaji/train")

# Generate predictions with enhanced function
sample_test["price"] = sample_test.apply(
    lambda row: predict_price(row["catalog_content"], row["image_link"]), axis=1
)

# Save predictions
sample_test[["sample_id", "price"]].to_csv("/balaji/train/sample_test_out_pred.csv", index=False)
print("Predictions saved to sample_test_out_pred.csv")


100%|██████████| 100/100 [00:00<00:00, 55916.60it/s]


Predictions saved to sample_test_out_pred.csv


In [21]:
import pandas as pd
import numpy as np

# Load predictions and actual values
pred_df = pd.read_csv("/balaji/train/sample_test_out_pred.csv")
actual_df = pd.read_csv("/content/sample_test_out.csv")

# Merge dataframes
merged_df = pred_df.merge(actual_df, on="sample_id", suffixes=("_pred", "_actual"))

def compute_smape(actual, predicted):
    n = len(actual)
    if n == 0:
        return np.nan
    absolute_error = np.abs(predicted - actual)
    denominator = (np.abs(actual) + np.abs(predicted)) / 2
    smape = np.mean(np.where(denominator != 0, absolute_error / denominator, 0)) * 100
    return smape

# Calculate SMAPE score
smape_score = compute_smape(merged_df["price_actual"], merged_df["price_pred"])
print(f"SMAPE Score on sample_test: {smape_score:.2f}%")

# Additional evaluation metrics
mae = np.mean(np.abs(merged_df["price_actual"] - merged_df["price_pred"]))
rmse = np.sqrt(np.mean((merged_df["price_actual"] - merged_df["price_pred"])**2))
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")


SMAPE Score on sample_test: 122.54%
MAE: 42.42
RMSE: 50.79
